# Preparations

### Import libraries

In [1]:
from pathlib import Path
from types import SimpleNamespace

from torch import nn

import lightning as pl
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.tuner import Tuner
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import wandb

from my_modules import MNISTDataModule, GenericModule

### Configuartion

In [2]:
path = Path.cwd() / "data"

In [3]:
CFG = SimpleNamespace()

# Data
CFG.BATCH_SIZE = 64
CFG.NUM_WORKERS = 8

# Model
CFG.INPUT_SIZE = 28*28
CFG.NUM_CLASSES = 10
CFG.LEARNING_RATE = 0.002
CFG.DROPOUT_RATE = 0.05

# Training
CFG.MAX_EPOCHS = 50

# Logging
CFG.PROJECT = "MNIST"

### Models

In [4]:
sequ_model_4_layer = nn.Sequential(
    nn.Linear(CFG.INPUT_SIZE, 80),
    nn.ReLU(),
    nn.Dropout(CFG.DROPOUT_RATE),
    nn.Linear(80, 40),
    nn.ReLU(),
    nn.Dropout(CFG.DROPOUT_RATE),
    nn.Linear(40, 20),
    nn.ReLU(),
    nn.Dropout(CFG.DROPOUT_RATE),
    nn.Linear(20, CFG.NUM_CLASSES),
    )

### Initialize

In [5]:
dm = MNISTDataModule(
    data_dir=path,
    batch_size=CFG.BATCH_SIZE,
    num_workers=CFG.NUM_WORKERS,
    )

In [6]:
module = GenericModule(
    model=sequ_model_4_layer,
    num_classes=CFG.NUM_CLASSES,
    learning_rate=CFG.LEARNING_RATE,
    )

In [7]:
wandb_logger = WandbLogger(project=CFG.PROJECT)

# Log hyperparameters
wandb_logger.experiment.config.update(vars(CFG))

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: th-ausserer (th-ausserer-personal-use). Use `wandb login --relogin` to force relogin


In [8]:
early_stopping = EarlyStopping(
    monitor='val_acc',
    min_delta=0.00,
    patience=3,
    verbose=False,
    mode='max',
    )

In [9]:
trainer = pl.Trainer(
    max_epochs=CFG.MAX_EPOCHS,
    logger=wandb_logger,
    callbacks=[early_stopping],
    )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Training

In [10]:
trainer.fit(module, dm)
wandb.finish()


  | Name     | Type               | Params
------------------------------------------------
0 | model    | Sequential         | 67.1 K
1 | loss_fn  | CrossEntropyLoss   | 0     
2 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
67.1 K    Trainable params
0         Non-trainable params
67.1 K    Total params
0.268     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇████
train_acc,▁▆▆▇▇▇▇▇█████████
train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇████
val_acc,▁▃▆▆▆▇▆██▇█▇████▇
val_loss,█▆▃▂▂▂▃▁▂▂▁▂▁▂▃▂▃
epoch,16
train_acc,0.98804
train_loss,0.04044
trainer/global_step,13293
val_acc,0.972


### Learning rate

In [ ]:
# Define tuner and find learning rate
tuner = Tuner(trainer)
lr_finder = tuner.lr_find(module, train_dataloaders=dm)

# Create plot with suggested learning rate
fig = lr_finder.plot(suggest=True)
fig.show()

# Hyperparameter search (sweep)

In [11]:
# Method for hyperparameter search
sweep_config = {
    'method': 'random'
    }

In [12]:
# Metric to optimize
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [13]:
# Parameters to search
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

{'method': 'random', 'metric': {'name': 'loss', 'goal': 'minimize'}}

# Debugging and testing